# Definable types as systems of equations

Our goal is to make pomagma.reducer smart enough to prove that `I:UNIT`, as in
definable_types.text (2016:08:23-25) (Q2) and
operational_semantics.text (2017:01:20-02:05) (Q5.A2.9):

<b>Desired Theorem:</b> `I : A \a,a',f,x. a(f(a' x))`, where

    copy := \x,y. x y y.
    join := \x,y,z. x(y|z).
    postconj := (\r,s. <B r, B s>).
    preconj := (\r,s. <CB s, CB r>).
    compose := (\r,s,r',s'. <r o r', s' o s>).
    A = A | <I, I> | <copy, join> | <div, BOT> | <BOT, TOP> | <C, C>
          | A preconj | A postconj | A (A compose).

In [1]:
from pomagma.reducer.lib import BOT, TOP, B, C, I, pair
from pomagma.reducer.sugar import as_term
from pomagma.reducer.syntax import NVAR, sexpr_print
from pomagma.reducer.systems import System, try_compute_step, try_decide_equal

We'll start with a definition of `A` by mutual recursion.

In [2]:
i = NVAR('i')
b = NVAR('b')
c = NVAR('c')
cb = NVAR('cb')
div = NVAR('div')
copy = NVAR('copy')
join = NVAR('join')
postconj = NVAR('postconj')
preconj = NVAR('preconj')
compose = NVAR('compose')
a = NVAR('a')

system_a = System(
    # Basic combinators.
    i=I,
    b=B,
    c=C,
    cb=C(B),
    # Components of A.
    div=(I | as_term(lambda x: div(x, TOP))),
    copy=as_term(lambda x, y: x(y, y)),
    join=as_term(lambda x, y, z: x(y | z)),
    postconj=as_term(lambda r, s: pair(b(r), b(s))),
    preconj=as_term(lambda r, s: pair(cb(s), cb(r))),
    compose=as_term(lambda r1, s1, r2, s2: pair(b(r1, r2), b(s2, s1))),
    # Definition of A, intended as a least fixed point.
    a=(pair(i,i) | pair(copy, join) | pair(div, BOT) | pair(c, c) |
       a(preconj) | a(postconj) | a(a(compose))
    ),
)
print(system_a.pretty())

       a = (JOIN (a postconj) (JOIN (a preconj) (JOIN (a (a compose)) (JOIN (ABS (0 c c)) (JOIN (ABS (0 copy join)) (JOIN (ABS (0 div BOT)) (ABS (0 i i))))))))
       b = (ABS (ABS (ABS (2 (1 0)))))
       c = (ABS (ABS (ABS (2 0 1))))
      cb = (ABS (ABS (ABS (1 (2 0)))))
 compose = (ABS (ABS (ABS (ABS (ABS (0 (b 4 2) (b 1 3)))))))
    copy = (ABS (ABS (1 0 0)))
     div = (JOIN (ABS 0) (ABS (div 0 TOP)))
       i = (ABS 0)
    join = (ABS (ABS (ABS (2 (JOIN 0 1)))))
postconj = (ABS (ABS (ABS (0 (b 2) (b 1)))))
 preconj = (ABS (ABS (ABS (0 (cb 1) (cb 2)))))


Next we'll extend this system to define a `UNIT` type.

A safer implementation of the type constructor `Simple` would apply `V` and check the input signature (e.g. here `unit_sig`). To keep the system simple, we'll directly apply `A` to the signature.

In [3]:
unit_sig = NVAR('unit_sig')
unit = NVAR('unit')

system_unit = system_a.extended_by(
    unit_sig=as_term(lambda r, s, f, x: r(f(s(x)))),
    unit=a(unit_sig),
)

Now we can try type checking

In [4]:
try:
    print(try_decide_equal(system_unit, i, unit(i)))
except NotImplementedError as e:
    print(e)

TODO handle JOIN: IVAR(0) vs JOIN(APP(APP(APP(APP(NVAR(a), NVAR(postconj)), NVAR(unit_sig)), NVAR(i)), IVAR(0)), JOIN(APP(APP(APP(APP(NVAR(a), NVAR(preconj)), NVAR(unit_sig)), NVAR(i)), IVAR(0)), JOIN(APP(APP(APP(APP(NVAR(unit_sig), NVAR(c)), NVAR(c)), NVAR(i)), IVAR(0)), JOIN(APP(APP(APP(APP(NVAR(unit_sig), NVAR(copy)), NVAR(join)), NVAR(i)), IVAR(0)), JOIN(APP(APP(APP(APP(NVAR(unit_sig), NVAR(div)), BOT), NVAR(i)), IVAR(0)), JOIN(APP(APP(APP(APP(NVAR(unit_sig), NVAR(i)), NVAR(i)), NVAR(i)), IVAR(0)), APP(APP(APP(APP(NVAR(a), APP(NVAR(a), NVAR(compose))), NVAR(unit_sig)), NVAR(i)), IVAR(0))))))))


Since the decision procedure is difficult to implement, let's instead try reducing.

In [5]:
def trace(system, name, steps=10):
    '''Print a reduction sequence'''
    system = system.copy()
    print(system.pretty())
    print('---- {} ----'.format(name))
    for step in xrange(steps):
        if not try_compute_step(system, name):
            print '[ Normalized ]'
            return
        print('{}: {}'.format(1 + step, sexpr_print(system[name])))
    print('[ Not Normalized ]')

In [6]:
system_i = system_unit.extended_by(unit_i=unit(i))

In [9]:
# This grows really quickly, and fails `py.test --nbval`.
if 0:
    trace(system_i, 'unit_i', 4)